# Исследование данных для развития игры "Секреты Темнолесья".

- Автор: Агафонова Ольга
- Дата: 26.03.2025г.

### Цели и задачи проекта

**Цель проекта** - изучение развития игровой индустрии с 2000г. по 2013г. на основе данных датасета `new_games.csv`.


**Задачи проекта:**

* Отбор и анализ данных за указанный период;
* Группировка данных по трём категориям, исходя из оценок пользователей и экспертов;
* Выделение ТОП-7 платформ по количеству игр, выпущенных за весь период.

### Описание данных

Данные датасета `new_games.csv`, которые будут задействованы в проктной работе, имеют следующее описание:


* `Name` — название игры;
* `Platform` — название платформы;
* `Year of Release` — год выпуска игры;
* `Genre` — жанр игры;
* `NA sales` — продажи в Северной Америке (в миллионах проданных копий);
* `EU sales` — продажи в Европе (в миллионах проданных копий);
* `JP sales` — продажи в Японии (в миллионах проданных копий);
* `Other sales` — продажи в других странах (в миллионах проданных копий);
* `Critic Score` — оценка критиков (от 0 до 100);
* `User Score` — оценка пользователей (от 0 до 10);
* `Rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

### Содержимое проекта

1. Загрузка данных и знакомство с ними.
2. Проверка ошибок в данных и их предобработка.
3. Фильтрация данных.
4. Категоризация данных.
5. Итоговый вывод.


---

## Загрузка данных и знакомство с ними

Загрузим необходимую библиотеку для анализа данных и данные из датасета new_games.csv. Затем выведем основную информацию о данных с помощью метода info() и первые строки датафрейма.


In [1]:
# Импортируем библиотеку pandas
import pandas as pd

In [2]:
# Выгружаем данные из датасета new_games.csv в датафрейм games
games = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')


In [3]:
# Выводим информацию о датафрейме
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [4]:
# Выводим первые строки датафрейма на экран
games.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN



Датафрейм games занимает объем оперативной памяти 1.4 МВ и содержит 16956 строк и 11 столбцов, в которых содержится информация о продажах компьютерных игр.

Изучим типы данных и их корректность:
В датасете представлены всего два типа данных: 


**Числовые значения с плавающей точкой (float64).** Четыре столбца представлены в данном типе: 

* `Year of Release` - год выпуска игры. Тип данных float64 не подходит для содержимого данного столбца, данные лучше представить в целочисленном формате (int64).
* `NA sales`, `Other sales` - содержит информацию о количестве проданных копий (в миллионах) в Северной Америке и других странах. Здесь вещественный тип данных подходит, так как значение может быть дробным числом.
* `Critic Score` - оценка критиков, в формате от 0 до 100. Тип float64 типичен для данных столбца.

**Строковые данные (object).** Семь столбцов имеют тип данных object:

* `Name`, `Platform`, `Genre`, `Rating`- столбцы имеют строковые значения (object), что вполне соответствует их содержимому: о названии игры и названии платформы, о жанре игры и рейтинге ESRB, который имеет буквенное или смешанное обозначение в зависимости от ограничений возрастной категории.  
* `EU sales`, `JP sales` - в столбцах указана информация о количестве проданных копий (в миллионах) в Европе и Японии. Эти данные могут быть представлены в виде дробных числовых значений, поэтому их нужно перевести в тип данных float64.
* `User Score` - столбец содержит оценку пользователей (от 0 до 10), которая также может быть дробным числом. Здесь логично будет присвоить значениям столбца тип данных float64.

Проанализировав типы данных видно, что большинство столбцов представлены корректно. А значения остальных столбцов необходимо привести к соответствующим типам данных.
Так же, в некоторых столбцах присутствуют пропуски, а именно: 
* `Name`- 2 пропуска,
* `Year of Release`- 275 пропусков,
* `Genre`- 2 пропуска,
* `Critic Score`- 8714 пропуска,
* `User Score`- 6804 пропуска,
* `Rating`- 6871 пропуск.

Пропущенным значениям необходимо уделить особое внимание. 

Так же необходимо обратить внимание на стиль написания названия столбцов. Они должны быть прописаны в нижнем регистре с разделением слов подчеркиваниями вместо пробелов, то есть в стиле snake case.


---

## Проверка ошибок в данных и их предобработка


### Названия, или метки, столбцов датафрейма

Выведем на экран названия всех столбцов датафрейма, проверим их стиль написания и приведем все столбцы датафрейма к стилю snake case. 


In [5]:
# Выведем все названия столбцов датафрейма games и проверим их стиль написания
print(games.columns)

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')


In [6]:
# Cоздадим список и сохраним в него новые названия столбцов датафрейма, приведенных к стилю snake case.
columns_snake_case = [
    'name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating']

# Передадим список атрибуту columns датафрейма
games.columns = columns_snake_case

# Проверим изменение названия столбцов
print(games.columns)


Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')


### Типы данных

Проведем преобразование некорректных типов данных.

In [7]:
# Преобразуем данные столбца 'year_of_release' типа float64 в целочисленный тип данных (int64). 
# Для этого заменим пропуски, например, на значения "-1":
games['year_of_release'] = games['year_of_release'].fillna(-1).astype('int64')

games.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [8]:
# Используем цикл для метода to_numeric(), чтобы преобразовать данные нескольких числовых столбцов типа object 
# в тип данных float64, заменив строковые "неизвестные значения" на пропуски NaN:
for column in ['eu_sales','jp_sales','user_score']:
    games[column] = pd.to_numeric(games[column], errors='coerce')
games.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16956 non-null  object 
 2   year_of_release  16956 non-null  int64  
 3   genre            16954 non-null  object 
 4   na_sales         16956 non-null  float64
 5   eu_sales         16950 non-null  float64
 6   jp_sales         16952 non-null  float64
 7   other_sales      16956 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       7688 non-null   float64
 10  rating           10085 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


### Наличие пропусков в данных

Считаем количество пропусков в каждом столбце в абсолютных и относительных значениях.


In [9]:
# Выводим абсолютное значение пропусков в столбцах датафрейма с помощью метода isna():
games.isna().sum()


name                  2
platform              0
year_of_release       0
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

После преобразований типов данных в столбцах количество пропусков изменилось. Отметим следующие столбцы датафрейма, которые остались с пропусками (их количество): `name`- **2**, `genre` - **2**, `eu_sales` - **6**, `jp_sales` - **4**, `critic_score` - **8714**, `user_score` - **9268** и `rating` - **6871**.

In [10]:
# Определим относительное значение пропусков (долю от общего числа строк):
games.isna().sum() / len(games) * 100

name                0.011795
platform            0.000000
year_of_release     0.000000
genre               0.011795
na_sales            0.000000
eu_sales            0.035386
jp_sales            0.023590
other_sales         0.000000
critic_score       51.391838
user_score         54.659118
rating             40.522529
dtype: float64

Анализируя столбцы с пропущенными значениями, можно сделать предварительный вывод:

1. Пропуски в столбцах `name` и `genre` вероятней всего случайны. Причина их возникновения не известна. Возможно, это техническая ошибка. Тип данных пропусков - MCAR. Они составляют менее 1%, следовательно, на анализ данных они не повлияют - их можно смело удалить.
2. Пропущенные значения в столбцах `eu_sales` и `jp_sales` можно объяснить, например, случайным отсутствием информации о продажах определенных компьютерных игр в данных регионах (определяются типом MAR). Их также менее 1%. Пропуски можно удалить, проигнорировать или заменить специальным средним значением. 
3. А вот пропуски в столбцах `critic_score`, `user_score` и `rating` нельзя проигнорировать, их доля в общем количестве строк составляет, примерно, 51%, 55% и 41% соответственно. Данный тип пропусков, вероятней всего, MNAR. Характер возникновения данных пропусков можно определить тем, что в `critic_score` и `user_score` оценка игры критиками и пользователями не указана, а в `rating` - либо у компьтерных игр, априори, отсутствует рейтинг организации ESRB, либо он не указан на сайте. Данные пропуски в столбцах можно заполнить или проигнорировать, как в случае со значениями в столбце `rating`, где, на мой взгдяд, невозможно самостоятельно подобрать рейтинг, поэтому пропуски NaN, которые больше характерны для типа данных 'float64', можно заменить текстовым значением **"unknown"**. Для столбцов `critic_score` и `user_score` можно использовать замещающее значение "-1", чтобы не исказить реальные данные по оценкам пользователей и критиков. 

In [11]:
# Удалим пропуски в столбцах `name` и `genre`, используя метод dropna():
games = games.dropna(subset=['name'])
games.isna().sum()

# Результат удаления показывает, что пропуски в столбце `name` соответствовали пропускам в столбце `genre`.
# Дополнительное удаление пропусков в столбце `genre` не требуется.

name                  0
platform              0
year_of_release       0
genre                 0
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8712
user_score         9266
rating             6869
dtype: int64

In [12]:
# Создадим функцию mean_sales, которая методом isna() выявит пропуски в столбце `eu_sales` и присвоит им среднее значение,  
# рассчитанное с помощью метода mean() относительно названия платформы и года выхода игры. 
# Условные параметры функции сохраним в переменной sales:
def mean_sales(row):
    if pd.isna(row['eu_sales']):
        sales = games[(games['platform'] == row['platform']) & (games['year_of_release'] == row['year_of_release'])]
# В случае обнаружения пропуска в столце 'eu_sales' функция подставит специальное среднее значение:
        return sales['eu_sales'].mean()
# В противном случае - вернёт имеющееся числовое значение рейтинга:
    else:
        return row['eu_sales']
        
# Заполним пропуски в столбце 'eu_sales', используя функцию mean_sales и метод apply():
games['eu_sales'] = games.apply(mean_sales, axis=1)

games.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [13]:
# Аналогично, используя созданную функцию mean_sales и методы isna() и mean(), проработаем пропуски в столбце `jp_sales`:
def mean_sales(row):
    if pd.isna(row['jp_sales']):
        sales = games[(games['platform'] == row['platform']) & (games['year_of_release'] == row['year_of_release'])]
        return sales['jp_sales'].mean()
    else:
        return row['jp_sales']
        
# Заполним пропуски по столбцу 'jp_sales', используя функцию mean_sales и метод apply():
games['jp_sales'] = games.apply(mean_sales, axis=1)

games.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [14]:
# Заменим пропущенные значения в столбцах `critic_score` и `user_score` на условное значение "-1":
games[['user_score', 'critic_score']] = games[['user_score', 'critic_score']].fillna(-1)

# А пропуски в столбце `rating` с данными строкового типа заменим на текстовое значение "unknown":
games['rating'] = games['rating'].fillna('unknown')

# Выведем результат:
games.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,-1.0,-1.0,unknown
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,-1.0,-1.0,unknown


Убедимся в отсутствии пропусков в столбцах датафрейма. 

In [15]:
# Выведем итоговый результат:
games.isna().sum()


name               0
platform           0
year_of_release    0
genre              0
na_sales           0
eu_sales           0
jp_sales           0
other_sales        0
critic_score       0
user_score         0
rating             0
dtype: int64

###  Явные и неявные дубликаты в данных


Проведем нормализацию данных. 

In [16]:
# Приведем текстовые данные столбцов `name`, `platform`, `genre` к нижнему регистру с помощью метода str.lower():
games['name'] = games['name'].str.lower()
games['platform'] = games['platform'].str.lower()
games['genre'] = games['genre'].str.lower()
games.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,wii sports,wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,super mario bros.,nes,1985,platform,29.08,3.58,6.81,0.77,-1.0,-1.0,unknown
2,mario kart wii,wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,wii sports resort,wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,pokemon red/pokemon blue,gb,1996,role-playing,11.27,8.89,10.22,1.00,-1.0,-1.0,unknown


In [17]:
# Столбец `rating` - к верхнему регистру с помощью метода str.upper():
games['rating'] = games['rating'].str.upper()
games.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,wii sports,wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,super mario bros.,nes,1985,platform,29.08,3.58,6.81,0.77,-1.0,-1.0,UNKNOWN
2,mario kart wii,wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,wii sports resort,wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,pokemon red/pokemon blue,gb,1996,role-playing,11.27,8.89,10.22,1.00,-1.0,-1.0,UNKNOWN


Избавимся от лишних пробелов в строковых столбцах.

In [18]:
# Используем метод pandas str.strip():
games['name'] = games['name'].str.strip()
games['platform'] = games['platform'].str.strip()
games['genre'] = games['genre'].str.strip()
games['rating'] = games['rating'].str.strip()
games.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,wii sports,wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,super mario bros.,nes,1985,platform,29.08,3.58,6.81,0.77,-1.0,-1.0,UNKNOWN
2,mario kart wii,wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,wii sports resort,wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,pokemon red/pokemon blue,gb,1996,role-playing,11.27,8.89,10.22,1.00,-1.0,-1.0,UNKNOWN


Проанализируем поочередно все уникальные значения столбцов `name`, `platform`, `genre` и `rating`, используя методы unique() и nunique() на выявление неявных дубликатов.



In [19]:
# Посчитаем количество уникальных значений столбца `name` в переменной unique_name_count:
unique_name_count = games['name'].nunique()

print(unique_name_count)

11559


Учитывая полученное количество уникальных строк (11559), выгружать и анализировать данные столбца `name` не представляется возможным. Перейдём к столбцу `platform`.

In [20]:
# Вычислим количество уникальных значений столбца `platform` в переменной unique_platform_count:
unique_platform_count = games['platform'].nunique()

print(unique_platform_count)

31


Количество уникальных значений в столбце `platform` - 31. С помощью метода unique() можно выгрузить массив уникальных данных и проанализировать их.

In [21]:
# Выведем все уникальные значения столбца `platform` в переменной unique_platform:
unique_platform = games['platform'].unique()

print(unique_platform)

['wii' 'nes' 'gb' 'ds' 'x360' 'ps3' 'ps2' 'snes' 'gba' 'ps4' '3ds' 'n64'
 'ps' 'xb' 'pc' '2600' 'psp' 'xone' 'wiiu' 'gc' 'gen' 'dc' 'psv' 'sat'
 'scd' 'ws' 'ng' 'tg16' '3do' 'gg' 'pcfx']


Названия платформ выглядят уникальными. Проанализируем жанры игр.

In [22]:
# Получим количество уникальных значений столбца `genre` в переменной unique_genre_count:
unique_genre_count = games['genre'].nunique()

print(unique_genre_count)

12


Изучим детально данные столбца `genre`.

In [23]:
# Выгрузим массив всех уникальных значений столбца `genre` в переменной unique_genre:
unique_genre = games['genre'].unique()

print(unique_genre)

['sports' 'platform' 'racing' 'role-playing' 'puzzle' 'misc' 'shooter'
 'simulation' 'action' 'fighting' 'adventure' 'strategy']


Все полученные значения столбца `genre` являются уникальными и замена значений не требуется. 

Перейдём к столбцу `rating`. Используя метод nunique(), выведем количество уникальных значений.

In [24]:
# Посчитаем количество уникальных значений столбца `rating` в переменной unique_rating_count:
unique_rating_count = games['rating'].nunique()

print(unique_rating_count)

9


Выгрузим названия возрастных рейтингов.

In [25]:
# Выведем все уникальные значения столбца `rating` в переменной unique_rating:
unique_rating = games['rating'].unique()

print(unique_rating)

['E' 'UNKNOWN' 'M' 'T' 'E10+' 'K-A' 'AO' 'EC' 'RP']


Из полученных значений видно, что рейтинг "К-А" в списке рейтингов организации ESRB не существует. Логично будет заменить его на значение рейтинга "Е" (Everyone - для всех).

In [26]:
# С помощью метода str.replace() заменим значение "К-А" на "Е":
games['rating'] = games['rating'].str.replace('K-A', 'E')

# Получим результат
display(games)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,wii sports,wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,super mario bros.,nes,1985,platform,29.08,3.58,6.81,0.77,-1.0,-1.0,UNKNOWN
2,mario kart wii,wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,wii sports resort,wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,pokemon red/pokemon blue,gb,1996,role-playing,11.27,8.89,10.22,1.00,-1.0,-1.0,UNKNOWN
...,...,...,...,...,...,...,...,...,...,...,...
16951,samurai warriors: sanada maru,ps3,2016,action,0.00,0.00,0.01,0.00,-1.0,-1.0,UNKNOWN
16952,lma manager 2007,x360,2006,sports,0.00,0.01,0.00,0.00,-1.0,-1.0,UNKNOWN
16953,haitaka no psychedelica,psv,2016,adventure,0.00,0.00,0.01,0.00,-1.0,-1.0,UNKNOWN
16954,spirits & spells,gba,2003,platform,0.01,0.00,0.00,0.00,-1.0,-1.0,UNKNOWN


Работа по выявлению **неявных** дубликатов проведена. Перейдём к устранению **явных** дубликатов в данных.

In [27]:
# Посчитаем количество строк до удаления дубликатов с помощью метода shape[0] и результат сохраним 
# в переменную initial_row_count:
initial_row_count = games.shape[0]

# Сортируем датафрейм по всем столбцам, используя метод sort_values():
games_sorted = games.sort_values(by=list(games.columns))

# Определим количество дубликатов методом duplicated():
duplicates = games_sorted[games_sorted.duplicated()].shape[0]

# Проведем удаление дубликатов в переменной games_no_duplicates с помощью метода drop_duplicates():
games_no_duplicates = games_sorted.drop_duplicates()

# Сохраним количество строк после удаления дубликатов в переменной final_row_count:
final_row_count = games_no_duplicates.shape[0]

# Выведем результат:
print(f'Количество строк до удаления дубликатов:{initial_row_count}')
print(f'Количество найденных дубликатов:{duplicates}')
print(f'Количество строк после удаления дубликатов:{final_row_count}')

Количество строк до удаления дубликатов:16954
Количество найденных дубликатов:241
Количество строк после удаления дубликатов:16713


Исходя из проведенных вычислений, сделаем промежуточный вывод:

1. Для определения количества дубликатов была проведена сортировка даных по всем столбцам. Далее, с помощью метода duplicated() вывели количество дубликатов по всему датафрейму - 241.
2. Провели удаление найденных дубликатов, используя метод drop_duplicates().

В процессе подготовки данных были удалены строки не только с дубликатами, но и с пропусками. Выведем общее количество удалённых строк в абсолютном и относительном значениях.

In [28]:
# Определим число всех удаленных строк в процессе предобработки данных и их относительное значение в процентном соотношении, 
# учитывая исходное количество строк - 16956:
total_row = 16956
del_row = total_row - final_row_count
part_del_row = round(((del_row / total_row) * 100), 2)
print(f'Исходное количество строк:{total_row}')
print(f'Общее количество удаленных строк:{del_row}')
print(f'Доля удаленных строк относительно общего количества:{part_del_row}%')

Исходное количество строк:16956
Общее количество удаленных строк:243
Доля удаленных строк относительно общего количества:1.43%


**Процесс оптимизации данных завершен. В итоге, было выявлено:**

1. Исходное количество строк - 16956 и 11 столбцов, из которых 7 столбцов с типом данных 'object' и 4столбца - 'float64'.


2. Столбцы `eu_sales`, `jp_sales`, `user_score` были преобразованы в тип данных 'float64', а `year_of_release` - в 'int64'.


3. В процессе предобработки данных датарейма всего было обнаржено 24867 пропущенных значений, с которыми была проведена следующая работа:
* в столбцах `name` и `genre` - 4 пропуска (2 строки), они удалены;
* в `year_of_release`, `critic_score` и `user_score` была проведена замена пропусков на условное значение "-1";
* пропуски в столбцах `eu_sales` и `jp_sales` были заполнены специальными расчетными средними значениями;
* а пропущенные значения столбца `rating` заменены текстовым значением "unknown".


4. Также была проведена работа по выявлению явных и неявных дубликатов, в ходе которой были проведены преобразования ряда строк датафрейма относительно анализа неявных дубликатов и проведено удаление явных дублкатов  количестве 241 строка.


5. Таким образом, для дальнейшего корректного анализа данных над строками были проведены все необходимые преобразования, в ходе которого всего было удалено 243 строки, что составляет порядка 1% от общего количества строк. 

---

## Фильтрация данных

Изучим историю продаж игр в период с 2000 по 2013 год включительно. Отберём данные по этому показателю. Сохраним новый срез данных в отдельном датафрейме `df_actual`.

In [29]:
# Отбираем строки датафрейма, которые соответствуют заданному периоду: 
df_actual = games[(games['year_of_release'] > 2000) & (games['year_of_release'] <= 2013)]

# Выводим информацию о новом датасете и первые его строки:
df_actual.info()
df_actual.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12623 entries, 0 to 16954
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             12623 non-null  object 
 1   platform         12623 non-null  object 
 2   year_of_release  12623 non-null  int64  
 3   genre            12623 non-null  object 
 4   na_sales         12623 non-null  float64
 5   eu_sales         12623 non-null  float64
 6   jp_sales         12623 non-null  float64
 7   other_sales      12623 non-null  float64
 8   critic_score     12623 non-null  float64
 9   user_score       12623 non-null  float64
 10  rating           12623 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 1.2+ MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,wii sports,wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E
2,mario kart wii,wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,wii sports resort,wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E
6,new super mario bros.,ds,2006,platform,11.28,9.14,6.50,2.88,89.0,8.5,E
7,wii play,wii,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E


Число строк в датасете `df_actual`, отражающем информацию о периоде с 2000 по 2013г., изменилось на 12623.

---

## Категоризация данных
    
Проведем категоризацию данных:
- Все игры за период с 2000г. по  2013г. разделим **по оценкам пользователей** и выделим такие категории: "высокая оценка" (от 8 до 10 включительно), "средняя оценка" (от 3 до 8, не включая правую границу интервала) и "низкая оценка" (от 0 до 3, не включая правую границу интервала).

In [30]:
import warnings
warnings.simplefilter(action='ignore')

# Проведём категоризацию по указанным параметрам с помощью метода cut() и сохраним изменения в новом столбце `user_category`:
df_actual['user_category'] = pd.cut(df_actual['user_score'], bins=[0, 3, 8, 10], labels=['низкая оценка', 'средняя оценка', 'высокая оценка'], right=False)

# Выведем первые строки датафрейма:
df_actual.head()


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_category
0,wii sports,wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E,высокая оценка
2,mario kart wii,wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E,высокая оценка
3,wii sports resort,wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E,высокая оценка
6,new super mario bros.,ds,2006,platform,11.28,9.14,6.50,2.88,89.0,8.5,E,высокая оценка
7,wii play,wii,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E,средняя оценка


- Разделим все игры данного периода **по оценкам критиков** и выделим такие категории: "высокая оценка" (от 80 до 100 включительно), "средняя оценка" (от 30 до 80, не включая правую границу интервала) и "низкая оценка" (от 0 до 30, не включая правую границу интервала).

In [31]:
import warnings
warnings.simplefilter(action='ignore')

# Выделим категории по заданным параметрам с помощью метода cut() и сохраним изменения в новом столбце `critic_category`:
df_actual['critic_category'] = pd.cut(df_actual['critic_score'], bins=[0, 30, 80, 100], labels=['низкая оценка', 'средняя оценка', 'высокая оценка'], right=False)

# Выведем первые строки датафрейма:
df_actual.head()



,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_category,critic_category
0,wii sports,wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E,высокая оценка,средняя оценка
2,mario kart wii,wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E,высокая оценка,высокая оценка
3,wii sports resort,wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E,высокая оценка,высокая оценка
6,new super mario bros.,ds,2006,platform,11.28,9.14,6.50,2.88,89.0,8.5,E,высокая оценка,высокая оценка
7,wii play,wii,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E,средняя оценка,средняя оценка


- После категоризации данных проверим результат: сгруппируем данные по выделенным категориям и посчитаем количество игр в каждой категории.

In [32]:
# Подсчитаем количество игр в каждой категории по оценкам пользователей, используя метод группировки groupby().
# Результат сохраним в переменную 'grouped_user_score': 
grouped_user_score = df_actual.groupby('user_category', as_index=False)['name'].count()

print(grouped_user_score)

    user_category  name
0   низкая оценка   115
1  средняя оценка  4094
2  высокая оценка  2256


In [33]:
# А в переменную 'grouped_critic_score' сохраним результат подсчета количества игр в каждой категории по оценкам критиков:
grouped_critic_score = df_actual.groupby('critic_category', as_index=False)['name'].count()

print(grouped_critic_score)

  critic_category  name
0   низкая оценка    52
1  средняя оценка  5397
2  высокая оценка  1674


Исходя из подсчета количества игр по столбцам `user_category` и `critic_category`, отметим, что общее число строк по каждому столбцу составляет 12623, что соответствует исходному значению строк датафрейма `df_actual`. Это значит, что при расчете учтены все данные датафрейма. По оценкам пользователей и критиков наибольшее количество игр принадлежат категории со средним оцениванием.

- Выделим топ-7 платформ по количеству игр, выпущенных за весь актуальный период.

In [34]:
# С помощью метода value_counts() определим число компьютерных игр относительно каждого уникального названия платформы
# и результат сохраним в переменную 'sorted_platform':
sorted_platform = df_actual['platform'].value_counts()

# Выведем ТОП-7 платформ с максимальным количеством игр:
print(sorted_platform.head(7))

# Определим долю выпущенных компьютерных игр в процентном соотношении на самых популярных платформах:  
print((df_actual['platform'].value_counts(normalize=True).head(7)) * 100)  


ds      2146
ps2     2072
wii     1294
psp     1199
x360    1138
ps3     1107
gba      825
Name: platform, dtype: int64
ds      17.000713
ps2     16.414482
wii     10.251129
psp      9.498534
x360     9.015290
ps3      8.769706
gba      6.535689
Name: platform, dtype: float64


* Основываясь на полученных данных и предпочтениях команды игры «Секреты Темнолесья», выведем компьтерные игры в жанре RPG, выпущенных на самой популярной платформе "ds", с высокими оценками пользователей и критиков, при условии, что количество проданных копий более одного миллиона. 

In [35]:
# Выведем популярные игры согласно условий, применив логическую маску. Результат сохраним в переменной 'actual':
actual = df_actual[(df_actual['critic_category'] == 'высокая оценка') & (df_actual['user_category'] == 'высокая оценка') & 
                   (df_actual['platform'] == 'ds') & (df_actual['genre'] == 'role-playing') &
                   ((df_actual['na_sales'] > 1) | (df_actual['jp_sales'] > 1) | (df_actual['eu_sales'] > 1) | (df_actual['other_sales'] > 1))]
display(actual.head())

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_category,critic_category
90,pokémon platinum version,ds,2008,role-playing,2.76,1.72,2.69,0.54,83.0,8.5,E,высокая оценка,высокая оценка
152,dragon quest ix: sentinels of the starry skies,ds,2009,role-playing,0.63,0.67,4.35,0.15,87.0,8.8,E10+,высокая оценка,высокая оценка
1193,dragon quest v: hand of the heavenly bride,ds,2008,role-playing,0.17,0.02,1.36,0.02,84.0,8.5,E10+,высокая оценка,высокая оценка


---

## Итоговый вывод

Анализируя всю проделанную работу с исходными данными датасета `new_games.csv`, подитожим:

1. Датасет `new_games.csv` содержит 11 столбцов и 16956 строк, которые содержат исторические данные о развитии всей игровой индустрии. При первичном знакомстве с данными и их предобработке были получены следующие результаты:

* для оптимизации работы с данными в датафрейме все названия столбцов были приведены к общему стилю snake case, а также были произведены некоторые изменения типов данных:
    - `eu_sales`, `jp_sales`, `user_score`: тип данных изменён с 'object' на 'float64';
    - `year_of_release`: тип данных изменён с 'float64' на 'int64'.
    
* в восьми столбцах (`name`, `genre`, `year_of_release`, `critic_score`, `user_score`, `eu_sales`, `jp_sales` и `rating`) были обнаружены пропущенные значения. Максимальные значения пропущенных данных в столбцах `critic_score`, `user_score` и `rating` — 51%, 55% и 41% соответственно. 

* в результате работы по выявлению явных и неявных дубликатов были проведены преобразования ряда строк датафрейма относительно анализа неявных дубликатов и удалены строки с явными дублкатами в количестве 241. Учитывая работу с пропусками, общее число удаленных строк составлет 243.

2. После предобработки данных приступили к анализу данных игровой индустрии за период с 2000 по 2013г. включительно, а именно:

* по оценкам пользователей и критиков игры были распределены на три категории: "низкая оценка", "средняя оценка" и "высокая оценка". Наибольшее число игр принадлежат категории среднего оценивания и по оценкам пользователей, и по оценкам критиков. Результаты категоризации были сожранены в новые столбцы `user_category` (категории оценки пользователей) и `critic_category` (категории оценки критиков).

* выведены ТОП-7 платформ с ниабольшим количеством выпущенных игр за указанный период. Подробнее (название платформы и кол-во игр): ds - 2146, ps2 - 2072, wii - 1294, psp - 1199, x360 - 1138, ps3 - 1107, gba - 825.

3. По предварительному анализу популярности компьютерных игр в жанре "ролевые игры", выпущенных, например, на самой востребованной платформе "ds", а также, учитывая высокие оценки критиков и пользователей и количество проданных копий (более 1 миллиона), можно рекомендовать команде игры "Секреты Темнолесья" в первую очередь обратить внимание на изучение развития следующих игр: **"pokémon platinum version"**, **"dragon quest ix: sentinels of the starry skies"** и **"dragon quest v: hand of the heavenly bride"**.  


    